# Regression 2 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
cal = pd.read_csv('cal_housing_clean.csv')

In [3]:
cal.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
cal.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [5]:
X_train, X_test, y_train, y_test = train_test_split(cal.drop('medianHouseValue',axis=1)
                                                    ,cal['medianHouseValue'],
                                                   random_state = 101, test_size=0.3)

In [6]:
X_train.shape,X_test.shape

((14448, 6), (6192, 6))

In [7]:
y_train.shape,y_test.shape

((14448,), (6192,))

### Scale the Feature Data

**Use sklearn preprocessing to create a MinMaxScaler for the feature data.**

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scale=MinMaxScaler()

**Fit this scaler only to the training data.**

In [10]:
scale.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

**Use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [12]:
X_train = pd.DataFrame(scale.transform(X_train),columns=X_train.columns,index=X_train.index)

In [13]:
X_test = pd.DataFrame(scale.transform(X_test),columns=X_test.columns,index=X_test.index)

In [14]:
X_train.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
6761,0.352941,0.069688,0.117163,0.048769,0.115442,0.142508
3010,0.607843,0.011242,0.015673,0.008367,0.014142,0.045027
7812,0.666667,0.025230,0.031347,0.020971,0.030258,0.212866
8480,0.666667,0.032530,0.033830,0.024752,0.030094,0.298651
1051,0.294118,0.031919,0.035692,0.019466,0.034863,0.272631


In [15]:
X_test.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome
16086,-0.006151,-0.000050,-0.000148,-0.000104,-0.000156,-0.010122
8816,-0.005767,-0.000050,-0.000152,-0.000105,-0.000161,0.018640
7175,-0.001922,-0.000050,-0.000149,-0.000104,-0.000157,-0.025260
16714,-0.013456,-0.000050,-0.000148,-0.000104,-0.000157,-0.016298
14491,-0.011534,-0.000049,-0.000144,-0.000104,-0.000153,0.011044


### Creating Feature Columns

**Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns.**

In [21]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 7 columns):
housingMedianAge    20640 non-null float64
totalRooms          20640 non-null float64
totalBedrooms       20640 non-null float64
population          20640 non-null float64
households          20640 non-null float64
medianIncome        20640 non-null float64
medianHouseValue    20640 non-null float64
dtypes: float64(7)
memory usage: 1.1 MB


In [17]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
households = tf.feature_column.numeric_column('households')
population = tf.feature_column.numeric_column('population')
income = tf.feature_column.numeric_column('medianIncome')

In [18]:
feat_cols = [age,rooms,bedrooms,population,households,income]

**Creating the input function for the estimator object. (we can play around with batch_size and num_epochs)**

In [23]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,
                                                 y=y_train,
                                                 batch_size=128,
                                                 num_epochs=200,
                                                shuffle=True)

**Create the estimator model. Using DNNRegressor.**

In [ ]:
model = tf.estimator.DNNRegressor(hidden_units=[10,])

##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

# Great Job!